In [1]:
print("Hello")

Hello


In [3]:
! pip install pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 55.4 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
# Reading a parquet file
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

taxi_df = pd.read_parquet(r'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

# 1. Total Column
print(taxi_df.shape)
# Column names
print(taxi_df.columns)

(3066766, 19)
Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')


In [5]:
# Calculating duration in minutes
# Converting to datetime
taxi_df['duration'] = taxi_df['tpep_dropoff_datetime'] - taxi_df['tpep_pickup_datetime']
display(taxi_df.head())

print(taxi_df['duration'].dt.total_seconds().std() / 60)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50


42.594351241954534


In [6]:
# Looking for outliers in duration and keeping only the rows with duration less than 60 minutes and greater than 1 minute
taxi_df_new = taxi_df.copy()

taxi_df_new = taxi_df_new[(taxi_df['duration'].dt.total_seconds() / 60 < 60) & (taxi_df_new['duration'].dt.total_seconds() / 60 > 1)]

# 3. Fraction of the rows that are kept
print(taxi_df_new.shape[0])
print(taxi_df.shape[0])
print((taxi_df_new .shape[0] / taxi_df.shape[0]) * 100)

3008849
3066766
98.11146334607858


In [ ]:
# One hot encoding on pickup and dropoff locations IDs.
# Turning the dataframe into a list of dictionaries and re-casting the IDs to strings
taxi_df_new['PULocationID'] = taxi_df_new['PULocationID'].astype(str)
taxi_df_new['DOLocationID'] = taxi_df_new['DOLocationID'].astype(str)

# Fitting a dictionary vectorizer to the pickup and dropoff location IDs
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

dv = DictVectorizer(sparse=False)
X = dv.fit_transform(taxi_df_new[['PULocationID', 'DOLocationID']].to_dict(orient='records'))
y = taxi_df_new['duration'].dt.total_seconds() / 60